# WhoScorred Match-Centre Scraping - Part 03

In this final part, I shall be cleaning the raw data that we scraped in the previous part and running some validation checks on this data.  

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
league_name = 'Premier League'
season_name = 2013

### Please import the excel file with the raw-scraped shots 
### data (named "PL 2013-14 -- Raw Shots Data") under the name "raw_shots_df"

filename = "2013-14.xlsx"
import_directory = "C:\\Users\\tharu\\OneDrive\\Desktop\\Big-PL-Project\\PL-Project\\WhoScored Shots Data - Raw\\Premier League\\"
import_path = os.path.join(import_directory, filename)

export_directory = "C:\\Users\\tharu\\OneDrive\\Desktop\\Big-PL-Project\\PL-Project\\WhoScored Shots Data - Final\\Premier League\\"
export_path = os.path.join(export_directory, filename)

raw_shots_df = pd.read_excel(import_path)

In [ ]:
### Seeing the list of columns present in the raw dataframe
raw_shots_df.columns

In [ ]:
### Listing out the total number of shots in the dataframe
len(raw_shots_df)

In [ ]:
### Looking at the unique entries currently present in the 'situation' column
print(raw_shots_df['situation'].unique())

In [ ]:
### Testing this 'situation' related columns for bogus entries
### These are the column names: 'shotOpenPlay', 'shotCounter', 'shotSetPiece' & 'shotDirectCorner'

situation_test_df = raw_shots_df
selected_columns_situation_test = ['minute', 'second', 'teamId', 'h_a', 'x', 'y', 'playerId', 
                                   'shotOpenPlay', 'shotCounter', 'shotSetPiece', 'shotDirectCorner']

situation_test_df = situation_test_df[selected_columns_situation_test].copy()

situation_test_df['dummy_shotOpenPlay'] = np.where(situation_test_df['shotOpenPlay'] == True, 1, 0)
situation_test_df['dummy_shotCounter'] = np.where(situation_test_df['shotCounter'] == True, 1, 0)
situation_test_df['dummy_shotSetPiece'] = np.where(situation_test_df['shotSetPiece'] == True, 1, 0)
situation_test_df['dummy_shotDirectCorner'] = np.where(situation_test_df['shotDirectCorner'] == True, 1, 0)

### Are there any shots that have 'True' entries for more than one of the above listed columns?
len(situation_test_df[(situation_test_df['dummy_shotOpenPlay'] + situation_test_df['dummy_shotCounter'] 
                   + situation_test_df['dummy_shotSetPiece']+ situation_test_df['dummy_shotDirectCorner']) > 1])

In [ ]:
### Grouping the shots by the 'situation' column (i.e. phase of play)
### The litmus test for this field - Normally, around 70% of all shots in a season are from open-play. Else, something's dodgy!
raw_shots_df.groupby(['situation'], dropna=False).size()

In [ ]:
### Cleaning up the [situation] column

cleaned_shots_df = raw_shots_df

def situation_cleaned_function(row):
    if pd.notnull(row['situation']):
        return row['situation']  
    elif pd.isnull(row['situation']) and row['shotCounter'] == True:
        return 'OpenPlay'
    elif pd.isnull(row['situation']) and row['shotSetPiece'] == True:
        return 'SetPiece'
    elif pd.isnull(row['situation']) and (row['penaltyScored'] == True or row['penaltyMissed'] == True):
        return 'Penalty'
    else:
        return np.nan

cleaned_shots_df['situation_cleaned'] = cleaned_shots_df.apply(situation_cleaned_function, axis=1)
print(cleaned_shots_df['situation_cleaned'].unique())

### Identifying the null entries in the new 'situation_cleaned' column (are they own-goals?)
(cleaned_shots_df[pd.isnull(cleaned_shots_df['situation_cleaned'])][['matchId', 'minute', 'second',  'playerName', 'goalOwn', 'home_teamId', 'away_teamId', 'start_date']])

In [ ]:
### Adding columns to identify the team our shots were registered for & against

def shot_taken_team_function(row):
    if row['goalOwn'] is True:
        if row['h_a'] == 'a':
            return row['home_teamId']
        elif row['h_a'] == 'h':
            return row['away_teamId']
    else:
        if row['h_a'] == 'h':
            return row['home_teamId']
        elif row['h_a'] == 'a':
            return row['away_teamId']
    return np.nan

def shot_conceded_team_function(row):
    if row['goalOwn'] is True:
        if row['h_a'] == 'a':
            return row['away_teamId']
        elif row['h_a'] == 'h':
            return row['home_teamId']
    else:
        if row['h_a'] == 'h':
            return row['away_teamId']
        elif row['h_a'] == 'a':
            return row['home_teamId']
    return np.nan
      
cleaned_shots_df['teamId_shot_for'] = cleaned_shots_df.apply(shot_taken_team_function, axis=1)
cleaned_shots_df['teamId_shot_against'] = cleaned_shots_df.apply(shot_conceded_team_function, axis=1)

In [ ]:
### Looking at the unique entries currently present in the 'type' column
print(raw_shots_df['type'].unique())

# These are the unique entries we want the final version of this column to have - 
# (MissedShot, SavedShot, Goal, ShotOnPost, OwnGoal, BlockedShot)

In [ ]:
### Testing these 'shot outcome' related columns for "bogus" entries
### These are the column names: 'shotOffTarget', 'shotOnPost', 'shotOnTarget','shotBlocked' & 'goalOwn'

outcome_test_df = raw_shots_df
selected_columns_outcome_test = ['minute', 'second', 'teamId', 'h_a', 'x', 'y', 'playerId', 
                                   'shotOffTarget', 'shotOnPost', 'shotOnTarget','shotBlocked', 'goalOwn']

outcome_test_df = outcome_test_df[selected_columns_outcome_test].copy()

outcome_test_df['dummy_shotOffTarget'] = np.where(outcome_test_df['shotOffTarget'] == True, 1, 0)
outcome_test_df['dummy_shotOnPost'] = np.where(outcome_test_df['shotOnPost'] == True, 1, 0)
outcome_test_df['dummy_shotOnTarget'] = np.where(outcome_test_df['shotOnTarget'] == True, 1, 0)
outcome_test_df['dummy_shotBlocked'] = np.where(outcome_test_df['shotBlocked'] == True, 1, 0)
outcome_test_df['dummy_goalOwn'] = np.where(outcome_test_df['goalOwn'] == True, 1, 0)

### Are there any shots that have 'True' entries for more than one of the above listed columns?
len(outcome_test_df[(outcome_test_df['dummy_shotOffTarget'] + outcome_test_df['dummy_shotOnTarget'] + 
                     outcome_test_df['dummy_shotBlocked']  + outcome_test_df['dummy_goalOwn']) > 1])

In [ ]:
### Are there any shots that hit the post which have not been classified as 'off target'?
outcome_test_df[(outcome_test_df['dummy_shotOffTarget'] == 0) & (outcome_test_df['dummy_shotOnPost'] == 1)]

In [ ]:
### Cleaning the 'type' column to create the afore-mentioned 'final' version
def type_cleaned_function(row):
    if row['shotBlocked'] == True:
        return 'BlockedShot'
    elif row['goalOwn'] == True:
        return 'OwnGoal'
    else:
        return row['type']

cleaned_shots_df['type_cleaned'] = cleaned_shots_df.apply(type_cleaned_function, axis=1)

### Listing out the unique entries of the new version of the column
print(cleaned_shots_df['type_cleaned'].unique())

### Searching any rows which have null values for this column
(cleaned_shots_df[pd.isnull(cleaned_shots_df['type_cleaned'])])

In [ ]:
### Testing shot location related columns for 'bogus' entries

location_test_df = raw_shots_df
selected_columns_location_test = ['minute', 'second', 'teamId', 'h_a', 'x', 'y', 'playerId', 
                                   'shotSixYardBox', 'shotPenaltyArea', 'shotOboxTotal']

location_test_df = location_test_df[selected_columns_location_test].copy()

location_test_df['dummy_shotSixYardBox'] = np.where(location_test_df['shotSixYardBox'] == True, 1, 0)
location_test_df['dummy_shotPenaltyArea'] = np.where(location_test_df['shotPenaltyArea'] == True, 1, 0)
location_test_df['dummy_shotOboxTotal'] = np.where(location_test_df['shotOboxTotal'] == True, 1, 0)

### Are there any shots that have 'True' entries for more than one of the above listed columns?
len(location_test_df[(location_test_df['dummy_shotSixYardBox'] + 
                     location_test_df['dummy_shotPenaltyArea']  + location_test_df['dummy_shotOboxTotal']) > 1])

In [ ]:
### Creating a single column for shot locations
def location_function(row):
    if row['shotSixYardBox'] == True:
        return 'Six Yard Box'
    elif row['shotPenaltyArea'] == True:
        return 'Penalty Area'
    elif row['shotOboxTotal'] == True:
        return 'Outside the Box'   
    elif row['x'] < 83 or (row['x'] >= 83 and row['y'] < 78.9) or (row['x'] >= 83 and row['y'] < 21.1):
        return 'Outside the Box'
    elif (row['x'] >= 94.2 and row['y'] >= 36.8 and row['y'] <= 63.2):
        return 'Six Yard Box'
    else:
        return 'Penalty Area'
              
cleaned_shots_df['Location'] = cleaned_shots_df.apply(location_function, axis=1)

### Listing out the unique entries of this column
print(cleaned_shots_df['Location'].unique())

### Searching any rows which have null values for this column
(cleaned_shots_df[pd.isnull(cleaned_shots_df['Location'])])

In [ ]:
### Adding in the team names to this dataframe using the WhoScored teamIds
### This is something that I manually obtained by going through each & every PL team's page on WhoScored for that season.


### I have imported the excel worksheet named "WhoScored-teamIds" three times under the names -
### "teamIds", "home_teamIds" and "away_teamIds". 

### First, getting the team names for the 'home_team' & 'away_team' columns
teamIds = pd.read_excel('C:\\Users\\tharu\\OneDrive\\Desktop\\Big-PL-Project\\PL-Project\\WhoScored Shots Data - Raw\\WhoScored-teamIds.xlsx')

home_teamIds = pd.read_excel('C:\\Users\\tharu\\OneDrive\\Desktop\\Big-PL-Project\\PL-Project\\WhoScored Shots Data - Raw\\WhoScored-teamIds.xlsx')
home_teamIds.rename(columns={'teamId': 'home_teamId'}, inplace=True)

away_teamIds = pd.read_excel('C:\\Users\\tharu\\OneDrive\\Desktop\\Big-PL-Project\\PL-Project\\WhoScored Shots Data - Raw\\WhoScored-teamIds.xlsx')
away_teamIds.rename(columns={'teamId': 'away_teamId'}, inplace=True)

cleaned_shots_df_teamId_join = cleaned_shots_df.merge(teamIds, on='teamId', how='left')
cleaned_shots_df_teamId_join.rename(columns={'team_name': 'team'}, inplace=True)
cleaned_shots_df_teamId_join.drop('team_country', axis=1, inplace=True)

cleaned_shots_df_home_teamId_join = cleaned_shots_df_teamId_join.merge(home_teamIds, on='home_teamId', how='left')
cleaned_shots_df_home_teamId_join.rename(columns={'team_name': 'home_team'}, inplace=True)
cleaned_shots_df_home_teamId_join.drop('team_country', axis=1, inplace=True)

cleaned_shots_df_away_teamId_join = cleaned_shots_df_home_teamId_join.merge(away_teamIds, on='away_teamId', how='left')
cleaned_shots_df_away_teamId_join.rename(columns={'team_name': 'away_team'}, inplace=True)
cleaned_shots_df_away_teamId_join.drop('team_country', axis=1, inplace=True)
cleaned_shots_df_away_teamId_join

In [ ]:
### Second, obtaining team names for the 'shots_for_teamId' & 'shots_against_teamId' columns
shot_for_teamIds = pd.read_excel('C:\\Users\\tharu\\OneDrive\\Desktop\\Big-PL-Project\\PL-Project\\WhoScored Shots Data - Raw\\WhoScored-teamIds.xlsx')
shot_for_teamIds.rename(columns={'teamId': 'teamId_shot_for'}, inplace=True)

shot_against_teamIds = pd.read_excel('C:\\Users\\tharu\\OneDrive\\Desktop\\Big-PL-Project\\PL-Project\\WhoScored Shots Data - Raw\\WhoScored-teamIds.xlsx')
shot_against_teamIds.rename(columns={'teamId': 'teamId_shot_against'}, inplace=True)

cleaned_shots_df_shot_for_team = cleaned_shots_df_away_teamId_join.merge(shot_for_teamIds, on='teamId_shot_for', how='left')
cleaned_shots_df_shot_for_team.rename(columns={'team_name': 'team_shot_for'}, inplace=True)
cleaned_shots_df_shot_for_team.drop('team_country', axis=1, inplace=True)

cleaned_shots_df_shot_against_team = cleaned_shots_df_shot_for_team.merge(shot_against_teamIds, on='teamId_shot_against', how='left')
cleaned_shots_df_shot_against_team.rename(columns={'team_name': 'team_shot_against'}, inplace=True)
cleaned_shots_df_shot_against_team.drop('team_country', axis=1, inplace=True)
cleaned_shots_df_shot_against_team

In [ ]:
### Adding the name of the league & the season
cleaned_shots_df_shot_against_team['League'] = league_name
cleaned_shots_df_shot_against_team['season'] = season_name

In [ ]:
### Removing unnecessary columns for the final shots dataframe & re-naming some of them
selected_columns = ['season', 'matchId', 'start_date', 'home_team', 'away_team', 
                 'minute', 'second', 'expandedMinute', 'team', 'h_a', 'x', 'y','period', 
                 'type_cleaned', 'playerId', 'playerName', 'shotBodyType',
                 'situation_cleaned', 'Location', 'relatedplayerName',
                 'blockedX', 'blockedY', 'goalMouthZ', 'goalMouthY', 
                 'team_shot_for', 'team_shot_against', 'League']
df = cleaned_shots_df_shot_against_team[selected_columns].copy()
df.rename(columns={
    'x': 'X',
    'y': 'Y',
    'type_cleaned': 'result',
    'shotBodyType': 'shotType',
    'situation_cleaned': 'situation',
    'relatedplayerName': 'player_assisted'}, inplace=True)
df

In [ ]:
### Performing final testing on the dataframe 
### Please cross-check with the WhoScored page for the 2013/14 PL season, unless otherwise stated.
### https://www.whoscored.com/regions/252/tournaments/2/seasons/3853/stages/7794/show/england-premier-league-2013-2014

print('There were', len(df[df['result'] != 'OwnGoal']), 'shots taken during the season (excluding own-goals)')
print('There were', len(df[df['result'] == 'OwnGoal']), 'own goals scored during the season')
print('There were', len(df[(df['situation'] == 'Penalty')]),
      'penalties taken during the season')

In [ ]:
### Grouping total shots by location
(df[df['result'] != 'OwnGoal']).groupby(['Location'])['Location'].count().sort_values(ascending=False)

In [ ]:
### Shots Taken Leaderboard (By Team)
(df[df['result'] != 'OwnGoal']).groupby(['team_shot_for'])['team_shot_for'].count().sort_values(ascending=False)

In [ ]:
### Goals scored leaderboard (By Team) - Cross-check with official PL table on their website!
df[(df['result'].isin(['Goal', 'OwnGoal']))].groupby(['team_shot_for'])['team_shot_for'].count().sort_values(ascending=False)

In [ ]:
### Goals conceded leaderboard (By Team) - Cross-check with official PL table on their website!
df[(df['result'].isin(['Goal', 'OwnGoal']))].groupby(['team_shot_against'])['team_shot_against'].count().sort_values(ascending=True)

In [ ]:
### Note that in WhoScored's 'Player Statistics' page for a season - the aggregation is done by player & club!
### So, keep this mind for players who moved clubs during the season in case the tallies you obtain here don't exactly match
### with the ones on WhoScored's list.

### Which players took the most shots during that season?
(df[df['result'] != 'OwnGoal']).groupby(['playerName'])['playerName'].count().sort_values(ascending=False).head(10)

In [ ]:
### Which players scored the most goals during the season?
(df[(df['result'] == 'Goal')]).groupby(['playerName'])['playerName'].count().sort_values(ascending=False).head(10)

In [ ]:
### Which players supplied the most assists during the season?
(df[(df['result'] == 'Goal')]).groupby(['player_assisted'])['player_assisted'].count().sort_values(ascending=False).head(10)

In [ ]:
#################################
### CONFIRMATION THAT I AM HAPPY WITH THE DATASET
#################################

In [ ]:
### Exporting the final/cleaned data to the final destination
df.to_excel(export_path, index=False)